# Volcano plots of PI3K/AKT/mTORi vs DMSO from the pseudobulkreplicates' differential gene expression analysese in the three models.

In [1]:
suppressWarnings({suppressMessages({
    library(EnhancedVolcano)
    library(ggplot2)
    library(ggpubr)
    library(dplyr)
})})

Loading the data.

In [2]:
JHOS2 <- readRDS(file = "JHOS2_PBRs_diffexpress.RDS")
PDC3 <- readRDS(file = "PDC3_PBRs_diffexpress.RDS")
PDC2 <- readRDS(file = "PDC2_PBRs_diffexpress.RDS")

Retaining only the results for the PI3K/AKT/mTOR inhibitors.

In [3]:
pi3k_akt_mtor <- c("AZD8055",
                   "AZD-8186",
                   "Buparlisib",
                   "Copanlisib",
                   "Dactolisib",
                   "Gedatolisib",
                   "Ipatasertib",
                   "NVP-BGT226",
                   "Pictilisib",
                   "TGX-221",
                   "ZSTK474")

Labelling specific genes of interest.

In [4]:
labelled_genes <- c("EGFR", "MET", "IRS1", "IRS2", "INSR", "SGK1", "CAV1", "FOXO3", "MYC")

Plotting the volcano plots.

In [5]:
plot_list_volcanos <- list()
for(drug in pi3k_akt_mtor){
    
    # 1. Extracting the data
    JHOS2_drug <- JHOS2[[drug]]
    PDC3_drug <- PDC3[[drug]]
    PDC2_drug <- PDC2[[drug]]
    
    # 2. Adding a column for the labels
    JHOS2_drug$label <- NA
    PDC3_drug$label <- NA
    PDC2_drug$label <- NA
    
    # 3. Putting the label to the genes of interest. These genes are present everywhere, but when
    # they are not significant, they will not get labelles
    JHOS2_drug[labelled_genes, "label"] <- labelled_genes
    PDC3_drug[labelled_genes, "label"] <- labelled_genes
    PDC2_drug[labelled_genes, "label"] <- labelled_genes
    
    # We remove the labels where there is no significance
    JHOS2_drug[JHOS2_drug$FDR > 0.01, "label"] <- NA
    PDC3_drug[PDC3_drug$FDR > 0.01, "label"] <- NA
    PDC2_drug[PDC2_drug$FDR > 0.01, "label"] <- NA
    
    # 4. Setting key values for the colors
    JHOS2_drug$up_down <- "no"
    PDC3_drug$up_down <- "no"
    PDC2_drug$up_down <- "no"
    
    JHOS2_drug[JHOS2_drug$logFC > 0 & JHOS2_drug$FDR < 0.01 & !is.na(JHOS2_drug$label), "up_down"] <- "up"
    JHOS2_drug[JHOS2_drug$logFC < 0 & JHOS2_drug$FDR < 0.01 & !is.na(JHOS2_drug$label), "up_down"] <- "down"
    
    PDC3_drug[PDC3_drug$logFC > 0 & PDC3_drug$FDR < 0.01 & !is.na(PDC3_drug$label), "up_down"] <- "up"
    PDC3_drug[PDC3_drug$logFC < 0 & PDC3_drug$FDR < 0.01 & !is.na(PDC3_drug$label), "up_down"] <- "down"
     
    PDC2_drug[PDC2_drug$logFC > 0 & PDC2_drug$FDR < 0.01 & !is.na(PDC2_drug$label), "up_down"] <- "up"
    PDC2_drug[PDC2_drug$logFC < 0 & PDC2_drug$FDR < 0.01 & !is.na(PDC2_drug$label), "up_down"] <- "down"
    
    # 5. Transforming to log10
    JHOS2_drug$log10FDR <- -log10(JHOS2_drug$FDR)
    PDC3_drug$log10FDR <- -log10(PDC3_drug$FDR)
    PDC2_drug$log10FDR <- -log10(PDC2_drug$FDR)
    
    # 6. Replacing infs with 350
    JHOS2_drug[JHOS2_drug$log10FDR == Inf, "log10FDR"] <- 350
    PDC3_drug[PDC3_drug$log10FDR == Inf, "log10FDR"] <- 350
    PDC2_drug[PDC2_drug$log10FDR == Inf, "log10FDR"] <- 350
    
    # 7. Plotting
    JHOS2_plot <- ggplot(JHOS2_drug, aes(y = log10FDR, 
                                 x = logFC, 
                                col = up_down,
                                size = up_down,
                                label = label))  +
                    geom_point(data = filter(JHOS2_drug, up_down == "no"), alpha = .02) + 
                    geom_point(data = filter(JHOS2_drug, !up_down == "no")) +
                    scale_size_manual(values = c("up" = 4, "down" = 4, "no" = 2)) +
                    scale_color_manual(values = c(
                        "up" = "firebrick",
                        "down" = "royalblue1",
                         "no" = "grey30"
                       )) +
                    geom_label_repel(max.overlaps = 100,
                        color = "black", fill= NA, box.padding = 1
                      ) + 
    ggtitle(paste0("JHOS2 ", drug)) +
    theme_classic() 
    
    PDC3_plot <- ggplot(PDC3_drug, aes(y = log10FDR, 
                                 x = logFC, 
                                col = up_down,
                                size = up_down,
                                label = label))  +
                    geom_point(data = filter(PDC3_drug, up_down == "no"), alpha = .02) + 
                    geom_point(data = filter(PDC3_drug, !up_down == "no")) +
                    scale_size_manual(values = c("up" = 4, "down" = 4, "no" = 2)) +
                    scale_color_manual(values = c(
                        "up" = "firebrick",
                        "down" = "royalblue1",
                         "no" = "grey30"
                       )) +
                    geom_label_repel(max.overlaps = 100,
                        color = "black", fill= NA, box.padding = 1
                      ) + 
    ggtitle(paste0("PDC3 ", drug)) +
    theme_classic()
    
    PDC2_plot <- ggplot(PDC2_drug, aes(y = log10FDR,
                                 size = up_down,
                                 x = logFC, 
                                col = up_down,
                                label = label))  +
                    geom_point(data = filter(PDC2_drug, up_down == "no"), alpha = .02) + 
                    geom_point(data = filter(PDC2_drug, !up_down == "no")) +
                    scale_size_manual(values = c("up" = 4, "down" = 4, "no" = 2)) +
                    scale_color_manual(values = c(
                        "up" = "firebrick",
                        "down" = "royalblue1",
                         "no" = "grey30"
                       )) +
                    geom_label_repel(max.overlaps = 100,
                        color = "black", fill= NA, box.padding = 1
                      ) + 
    ggtitle(paste0("PDC2 ", drug)) +
    theme_classic()
    
    # Saving individually and as a whole
    ggsave(plot = ggarrange(plotlist = list(PDC2_plot, PDC3_plot, JHOS2_plot), ncol = 3), 
           filename = paste0(drug, "_PBRs_VolcanoPlot.pdf"), width = 20, height = 5)

    plot_list_volcanos[[paste0("JHOS2 ", drug)]] <- JHOS2_plot
    plot_list_volcanos[[paste0("PDC3 ", drug)]] <- PDC3_plot
    plot_list_volcanos[[paste0("PDC2 ", drug)]] <- PDC2_plot
}

Warning message:
“Removed 14108 rows containing missing values (`geom_label_repel()`).”
Warning message:
“Removed 14361 rows containing missing values (`geom_label_repel()`).”
Warning message:
“Removed 12953 rows containing missing values (`geom_label_repel()`).”
Warning message:
“Removed 14014 rows containing missing values (`geom_label_repel()`).”
Warning message:
“Removed 14343 rows containing missing values (`geom_label_repel()`).”
Warning message:
“Removed 12885 rows containing missing values (`geom_label_repel()`).”
Warning message:
“Removed 14117 rows containing missing values (`geom_label_repel()`).”
Warning message:
“Removed 14396 rows containing missing values (`geom_label_repel()`).”
Warning message:
“Removed 12957 rows containing missing values (`geom_label_repel()`).”
Warning message:
“Removed 14123 rows containing missing values (`geom_label_repel()`).”
Warning message:
“Removed 14389 rows containing missing values (`geom_label_repel()`).”
Warning message:
“Removed 12986 

Plotting all the volcanos in the same PDF too.

In [6]:
ggsave(filename = "PI3K.AKT.mTORi_PBRs_VolcanoPlots.pdf", width = 20, height = 50, limitsize = FALSE,
       plot = ggarrange(plotlist = plot_list_volcanos, ncol = 3, nrow = 11))

Warning message:
“Removed 12953 rows containing missing values (`geom_label_repel()`).”
Warning message:
“Removed 14361 rows containing missing values (`geom_label_repel()`).”
Warning message:
“Removed 14108 rows containing missing values (`geom_label_repel()`).”
Warning message:
“Removed 12885 rows containing missing values (`geom_label_repel()`).”
Warning message:
“Removed 14343 rows containing missing values (`geom_label_repel()`).”
Warning message:
“Removed 14014 rows containing missing values (`geom_label_repel()`).”
Warning message:
“Removed 12957 rows containing missing values (`geom_label_repel()`).”
Warning message:
“Removed 14396 rows containing missing values (`geom_label_repel()`).”
Warning message:
“Removed 14117 rows containing missing values (`geom_label_repel()`).”
Warning message:
“Removed 12986 rows containing missing values (`geom_label_repel()`).”
Warning message:
“Removed 14389 rows containing missing values (`geom_label_repel()`).”
Warning message:
“Removed 14123 

In [7]:
sessionInfo()

R version 4.2.2 (2022-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Rocky Linux 8.8 (Green Obsidian)

Matrix products: default
BLAS/LAPACK: /homedir01/adini22/.conda/envs/cellhashing_analyses/lib/libopenblasp-r0.3.21.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] dplyr_1.0.10           ggpubr_0.5.0           EnhancedVolcano_1.16.0
[4] ggrepel_0.9.3          ggplot2_3.4.2         

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.10       pillar_1.9.0      compiler_4.2.2    base64enc_0.1-3  
 [5] tools_4.2.2       digest_